In [6]:
import numpy as np
import h5py

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import scipy as scp

import skimage.exposure
from skimage.io import imread
from skimage.filters import *
from skimage.morphology import *
from skimage import data
from skimage import img_as_float
from skimage.morphology import disk
from skimage import measure

# Image coordinates are
# top-left corner is (0,0)  and (10,4) addresses pixel on the 10th row and 4th colum

pxPerMeter= 967.5931
expnr=5
images = []
dt=0.002;

background = img_as_float(imread('../../data/experiment%i/exp%i-0001.jpg' % (expnr,expnr), as_grey = True))

mask = np.ones(background.shape,np.uint8)
bounds = [0.0,background.shape[1]/pxPerMeter,0.0,background.shape[0]/pxPerMeter]

r = range(536,538)
count = 0
for i in r:
    
    frame = '../../data/experiment%i/exp%i-%04i.jpg' % (expnr,expnr,i)
    print("Process Frame: ", frame)
    
    image =  img_as_float(imread(frame, as_grey = True))
    #images.append(image)


    # testing image processing stuff =================================================
    image_clahe = skimage.exposure.equalize_adapthist(image,ntiles_x=16,ntiles_y=16);
    
    # background subtract and make binary mask
    subtracted = np.abs(image - background)
    thresh = threshold_otsu(subtracted)
    binary = (subtracted > thresh).astype(np.uint8)
    d = disk(2)
    binary = dilation(binary, d)
    
    # only thresholding
    thresh = threshold_otsu(subtracted)
    binary2 = subtracted > 0.5*thresh
    
    # find max. length contour (interpolate)
    contours = measure.find_contours(binary2, 0.5)
    if contours :
        contourMax = max(contours,key=lambda x: x.shape[0])
        tck,u = scp.interpolate.splprep([contourMax[:, 0], contourMax[:, 1]], s=1e4)
        unew = np.linspace(0, 1.0, 3000)
        contourInterp = scp.interpolate.splev(unew, tck)
    else:
        contourMax = None
        contourInterp = None
    # =================================================================================
    
    
    # Plotting ========================================================================
#     fig, axes = plt.subplots(ncols=3,nrows=3, figsize=(15, 8), sharex=True, sharey=True)
#     fig.set_tight_layout(True)
#     i = 1
#     for row in axes:
#         for a in row:
#             exec("ax%i=a" % i)
#             i+=1
    
    #plt.matshow( image_clahe , cmap=cm.Greys_r );
    
#     ax1.imshow(image,cmap=cm.Greys_r)
#     ax1.set_title('original image')
#     ax1.axis('off')
    
#     ax2.imshow(binary,cmap=cm.Greys_r)
#     ax2.set_title('background subtracted binary mask')
#     ax2.axis('off')

    
# #         image_g = gaussian_filter(image, 2)
# #         seed = np.copy(image_g)
# #         seed[1:-1, 1:-1] = image_g.min()
# #         mask = image_g
# #         dilated = reconstruction(seed, mask, method='dilation')

# #     ax2.imshow(dilated, vmin=image.min(), vmax=image.max(),cmap=cm.Greys_r)
# #     ax2.set_title('dilated')
# #     ax2.axis('off')
    
# #     subtracted = image - dilated
# #     ax2.imshow(subtracted,cmap=cm.Greys_r)
# #     ax2.set_title('image - dilated')
# #     ax2.axis('off')
    
#     ax3.imshow(binary2,cmap=cm.Greys_r)
#     ax3.set_title('thresholded (otsu)')
#     ax3.axis('off')


#     for n, contour in enumerate(contours):
#         ax4.plot(contour[:, 1], contour[:, 0], linewidth=0.5)
#     #ax3.plot(contourMax[:,1], contourMax[:,0], "bo", linewidth=2 )   
#     ax3.plot(contourInterp[1], contourInterp[0], "r", linewidth=2)    
    # =================================================================================
    
    
    # Make Countour Frames
#     dpi=100
#     f = plt.figure(num=None, frameon=False, figsize=(background.shape[1]/dpi,background.shape[0]/dpi), dpi=dpi)
#     ax = plt.Axes(f, [0., 0., 1., 1.])
#     #ax.set_axis_off()
#     f.add_axes(ax)
#     ax.imshow(image,cmap=cm.Greys_r)
#     ax.set_autoscalex_on(False)
#     ax.set_autoscaley_on(False)
#     if contourInterp:
#           ax.plot(contourInterp[1], contourInterp[0], "b", linewidth=2)
#     f.savefig("Contour%04i.jpg" % count)
#     plt.close(f)
#     f=None
#     ax=None
    
    time = count*dt;
    h5File = h5py.File('image%03i.h5'%count, 'w')
    dataset = h5File.create_dataset("bounds", data=np.array(bounds))
    dataset = h5File.create_dataset("data", data=image_clahe)
    dataset = h5File.create_dataset("mask", data=mask)
    dataset = h5File.create_dataset("time", data=np.array(time))
    h5File.close()

    
    count+=1
    
#plt.show()

Process Frame:  ../../data/experiment5/exp5-0536.jpg
Process Frame:  ../../data/experiment5/exp5-0537.jpg
